<a href="https://colab.research.google.com/github/ElnathanTiokou/Adversarial_Robustness_of_FL_systems/blob/main/Tensorflow_federated_Robustness/executable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install --quiet  tensorflow-federated==0.20.0   # The latest version of tensorflow-federated is not working with the colab python version
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()


import tensorflow as tf
import tensorflow_federated as tff

from attacks.data_attacker import LabelAttacker, NoiseMutator, OverlapMutator, DeleteMutator, UnbalanceMutator
from config import get_model, load_data, get_num_round, get_param, throw_conf_error, get_result_dir
from fedPipeline import FedPipeline
from attacks.model_attacker import SignFlipModelAttacker, BackdoorAttack, RandomModelAttacker
from Dataset.dataset import Dataset
import numpy as np



emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

# In order to feed the dataset into our model, we flatten the data, and convert each example into a tuple of the form: flatten image and label.

NUM_CLIENTS = 100    # Selected number of clients: simulation of a cross-device
BATCH_SIZE = 20



dataset = Dataset()


def create_model():

    initializer = tf.keras.initializers.GlorotNormal(seed=0)
    keras_model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(784,)),
        tf.keras.layers.Dense(10, kernel_initializer=initializer),
        tf.keras.layers.Softmax(),
    ])
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

def get_model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


def get_attacks():
    attack_map = {}
    attack = get_param('attack', 'none')
    if attack == 'none':
        return attack_map
    fraction = get_param('attack-fraction')
    if attack == 'label-flip':
        attack_map['data_attacker'] = LabelAttacker(fraction)
    elif attack == 'noise-data':
        attack_map['data_attacker'] = NoiseMutator(fraction)
    elif attack == 'overlap-data':
        attack_map['data_attacker'] = OverlapMutator(fraction)
    elif attack == 'delete-data':
        attack_map['data_attacker'] = DeleteMutator(fraction)
    elif attack == 'unbalance-data':
        attack_map['data_attacker'] = UnbalanceMutator(fraction)
    elif attack == 'random-update':
        attack_map['model_attacker'] = RandomModelAttacker(fraction, 2)
    elif attack == 'sign-flip':
        attack_map['model_attacker'] = SignFlipModelAttacker(fraction, 10)
    elif attack == 'backdoor':
        attack_map['model_attacker'] = BackdoorAttack(fraction, 1)
    else:
        throw_conf_error('attack')
    return attack_map

fed_tester = FedPipeline(
    get_model_fn(),
    dataset,
    **get_attacks()
)

results = fed_tester.perform_fed_training(get_num_round())

results_dir = get_result_dir()
for key in results:
    file_path = results_dir + key
    result = results[key]
    if key == 'time':
        with open(file_path, 'w') as file:
            file.write(str(result))
            file.close()
    else:
        np.save(file_path, np.array(result))